## CEX-DEX Arbitrage Research

Use this Notebook as the starting point for your CEX-DEX arbitrage research.

In [1]:
import os
import asyncio
import nest_asyncio
import aioprocessing
from functools import partial
from dotenv import load_dotenv

load_dotenv(override=True)

HTTP_RPC_URL = os.getenv('HTTP_RPC_URL')
WS_RPC_URL = os.getenv('WS_RPC_URL')

#### Utils

A function for handling websocket connection errors. Wrapping data streaming functions with **reconnecting_websocket_loop** will automatically reconnect everytime a connection error occurs.

In [2]:
from utils import (
    reconnecting_websocket_loop,
    calculate_next_block_base_fee,
)

#### CEX data streams

Currently implemented for Binance websocket stream of depth5 data.

In [3]:
from cex_streams import (
    stream_binance_usdm_orderbook,
    stream_okx_usdm_orderbook,
)

#### DEX data streams

In [4]:
from dex_streams import (
    stream_new_blocks,
    stream_uniswap_v2_events,
)

#### Aggregator

In [5]:
from aggregator import aggregate_cex_orderbooks

#### Event handler

In [6]:
import datetime
from spread_chart import Publisher

"""
Using taker fee to make spread calculations more realistic (Tier: LV 1)

- OKX: https://www.okx.com/fees
- Binance: https://www.binance.com/en/fee/futureFee
"""
FEE = {
    'okx': 0.0005,      # 0.05%
    'binance': 0.0004,  # 0.04%
}

async def cex_event_handler(port: int, event_queue: aioprocessing.AioQueue):   
    pub = Publisher(port)

    orderbooks = {}
    
    last_updated = datetime.datetime.now()
    
    while True:
        data = await event_queue.coro_get()
        
        symbol = data['symbol']
        
        if data['source'] == 'cex':
            if symbol not in orderbooks:
                orderbooks[symbol] = {}
                
            orderbooks[symbol][data['exchange']] = data
            multi_orderbook = aggregate_cex_orderbooks(orderbooks[symbol])
            
            now = datetime.datetime.now()
            if (now - last_updated).total_seconds() > 1:
                best_bid = multi_orderbook['bids'][0][0]
                best_ask = multi_orderbook['asks'][0][0]
                bid_ask_spread = round(float((best_bid / best_ask - 1) * 100), 3)
                
                best_bid_exchange = multi_orderbook['bids'][0][2]
                best_ask_exchange = multi_orderbook['asks'][0][2]
                target_exchanges = f'{best_bid_exchange}/{best_ask_exchange}'
                
                bid_fee = FEE[best_bid_exchange] * 2 * 100  # buy, sell fee (x2)
                ask_fee = FEE[best_ask_exchange] * 2 * 100  # buy, sell fee (x2)
                bid_ask_spread_real = bid_ask_spread - (bid_fee - ask_fee)
                
                await pub.send({'spread': bid_ask_spread_real})
                
                last_updated = now
                print(f'[{now}] Spread: {bid_ask_spread_real}% ({target_exchanges})')

#### Run streams

In [7]:
from threading import Thread

def event_handler_loop(port: int, event_queue: aioprocessing.AioQueue):
    asyncio.run(cex_event_handler(port, event_queue))

# define an event_queue to publish realtime data
event_queue = aioprocessing.AioQueue()

symbols = ['ETH/USDT']

binance_stream = reconnecting_websocket_loop(
    partial(stream_binance_usdm_orderbook, symbols, event_queue),
    tag='binance_stream'
)

okx_stream = reconnecting_websocket_loop(
    partial(stream_okx_usdm_orderbook, symbols, event_queue),
    tag='okx_stream'
)
    
# start event_handler_loop in a new thread
chart_port = 9999

t = Thread(target=event_handler_loop, args=(chart_port, event_queue,))
t.start()

nest_asyncio.apply()

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait([
    binance_stream,
    okx_stream,
]))

[2023-08-08 18:29:13.845864] Spread: 0.058% (okx/binance)
[2023-08-08 18:29:14.860689] Spread: 0.058% (okx/binance)
[2023-08-08 18:29:15.942755] Spread: 0.058% (okx/binance)
[2023-08-08 18:29:17.026610] Spread: 0.058% (okx/binance)
[2023-08-08 18:29:18.050501] Spread: 0.058% (okx/binance)
[2023-08-08 18:29:19.184212] Spread: 0.058% (okx/binance)
[2023-08-08 18:29:20.208681] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:21.253464] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:22.266899] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:23.322959] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:24.356917] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:25.430344] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:26.448566] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:27.448862] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:28.502045] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:29.557245] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:30.571954] Spread: 0.062% (okx/binance)
[2023-08-08 18

[2023-08-08 18:29:37.786373] Spread: 0.062% (okx/binance)
[2023-08-08 18:29:38.862983] Spread: 0.062% (okx/binance)
